In [ ]:
# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/home/orco/Dokumente/PTB/MRS/MRSproc')


In [ ]:
from mrsproc.preprocess import *
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import glob
import os
import suspect
import multiprocessing as mp
plt.rcParams['figure.figsize'] = [10, 7]

In [ ]:
mm_files = glob.glob('/home/orco/Dokumente/PTB/MRS/MacroMols_7T/sub-*/mrs/sub-*_nuc-1H_loc-*_spec-ad-special*.dat')

In [ ]:
len(mm_files)

In [ ]:
mm_files

In [ ]:
from joblib import Parallel, delayed
n_cpu = 4

In [ ]:
%%time
with Parallel(n_jobs=2) as p:
    raw = p(delayed(suspect.io.load_twix)(i) for i in mm_files)

In [ ]:
raw[0].shape

In [ ]:
# The last twos are the sub.701 32 agerages each, they have to be merged
a1 = raw.pop(-1)
a2 = raw.pop(-1)

In [ ]:
a = np.append(a1, a2, axis=0)

In [ ]:
a.shape

In [ ]:
amrs = raw[0].inherit(a)

In [ ]:
(amrs == a).all()

In [ ]:
raw = raw + [amrs]

In [ ]:
# Average every odd and even acquisition

In [ ]:
def mean_everytwo(mm1):
    out = 0.5*(mm1[0::2] + mm1[1::2])
    return mm1.inherit(out)

In [ ]:
%%time
with Parallel(n_jobs=2) as p:
    mm2 = p(delayed(mean_everytwo)(i) for i in raw)

In [ ]:
[i.shape for i in mm2]

In [ ]:
def coil_combine(mrs):
    coil_combined = []
    for i in range(mrs.shape[0]):
        weights = suspect.processing.channel_combination.svd_weighting(mrs[i,:,:])
        coil_combined.append(suspect.processing.channel_combination.combine_channels(mrs[i,:,:],weights))
    return mrs.inherit(np.array(coil_combined))

In [ ]:
%%time
coil_combined = [coil_combine(i) for i in mm2]

In [ ]:
[i.shape for i in coil_combined]

In [ ]:
[type(i) for i in coil_combined]

In [ ]:
first = coil_combined[0]
first.shape

In [ ]:
first.frequency_axis_ppm()

In [ ]:
first = coil_combined[0]
plt.plot(first.frequency_axis_ppm(), first.spectrum().real)
plt.xlim([5,0])
plt.axvline(4.7)

In [ ]:
corr = suspect.processing.frequency_correction.residual_water_alignment(first)
first = first.adjust_frequency(-corr)
corr